In [1]:
import xarray as xr
import numpy as np
import glob
import os.path

from dask.distributed import Client
from dask.diagnostics import ProgressBar

from aggfly import dataset, regions, grid_weights
from aggfly.aggregate import TemporalAggregator, SpatialAggregator, get_time_dim

ProgressBar().register()
# client = Client()

In [3]:
# Set file output name/path
output_path = "/home3/dth2133/data/aggregated/counties/"
crop = 'soy'
output_name = f"usa_counties_{crop}_monthly"
output_varn = "prec"
csv = True

In [4]:
# Open shapefile containing region features.
georegions = regions.from_name('counties')

# Open example climate dataset to calculate grid weights.
clim = dataset.from_path(
    f"/home3/dth2133/data/usa/usa-tp_tempPrecLand2019.zarr", 
    'tp', 
    'zarr', 
    preprocess=dataset.timefix_era5l)

# Clip climate data to the US (raw data are global)
clim.clip_data_to_georegions_extent(georegions)

# Rechunk dataset to optimize multithreading
clim.rechunk((5, 578, -1, -1, -1, -1))

# Calculate area and crop layer weights.
weight_dict = dict(
    corn='corn',
    soy='soyb',
    cotton='cott')
weights = grid_weights.from_objects(clim, georegions, crop=weight_dict[crop])

# This object covers aggregating hourly and daily data to the yearly 
# level
# daily = [TemporalAggregator(
#     'dd',
#     agg_from='hour',
#     agg_to='day', 
#     ddargs=[x,999,0]) for x in range(-3,36)]

annual = TemporalAggregator(
    'sum', 
    agg_from='hour',
    agg_to='month')

# This object aggregates cells within a region to the average across 
# cells, weighted by `weights`, which in this case are the area of the
# cell and the share of the cell with corn crops.
spatial = SpatialAggregator('avg')


In [5]:
# Calculate the grid weights
w = weights.weights()

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 25.6s
[########################################] | 100% Completed | 25.6s
[########################################] | 100% Completed | 42.6s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


In [26]:
# import importlib
# import aggfly
# from aggfly.aggregate.temporal import TemporalAggregator
# importlib.reload(aggfly.aggregate.temporal)
# from aggfly.aggregate.temporal import TemporalAggregator

In [6]:
import dask
def timefix_era5l_tp(array):
    with dask.config.set(**{'array.slicing.split_large_chunks': False}):   
        array['year'] = array.time.dt.year
        array['month'] = array.time.dt.month
        array['day'] = array.time.dt.day
        array['hour'] = array.time.dt.hour
        array = array.set_index(time=("year", "month", "day", "hour")).unstack('time')
        # array = array - 273.15
    return array

In [7]:
def aggregate_era5l_tp(path):
    # path = "/home3/dth2133/data/usa/usa-t2m_tempPrecLand1969.zarr"
    # Open climate dataset.
    clim = dataset.from_path(
        path, 
        'tp', 
        'zarr', 
        preprocess=timefix_era5l_tp) # Kelvin to Celsius
    # print(clim)
    # Clip climate data to the US (raw data are global)
    clim.clip_data_to_georegions_extent(georegions)
    # Rechunk dataset to optimize multithreading
    clim.rechunk((5, 578, -1, -1, -1))

    # Update climate dataset in `clim` to one collapsed over
    # hour and day based upon `temporal` definition above.
    # clim = daily.map_execute(clim)
    annual.map_execute(clim, update=True)
    clim.rechunk(-1)

    # Again update, but with (weighted) spatial collapse over regions.
    spatial.map_execute(clim, w)

    # Send back the aggregated climate data.
    return clim.da


In [8]:
# Check years from the input path - this just creates a vector of years for
# which my raw climate data are available, e.g. [1970, 1971, ...]
import numpy as np
import glob
from os.path import basename
files = np.sort([x for x in glob.glob('/home3/dth2133/data/usa/*tp_*')])
# Loop over years and aggregate.
output = list()
for f in files:
    print(f)
    output.append(aggregate_era5l_tp(f))

/home3/dth2133/data/usa/usa-tp_tempPrecLand1951.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1952.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1953.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1954.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1955.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1956.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1957.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1958.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1960.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1961.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1962.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1963.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1964.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1965.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1966.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1967.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1968.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand1969.zarr
/home3/dth2133/data/usa/usa-tp_tempPrecLand197

In [9]:
da = xr.concat(output, dim='year').compute()

[########################################] | 100% Completed | 31min 52.7s


In [18]:
outds = da.to_dataset(name=output_varn)

In [19]:
ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))
# ds = ds.drop(output_varn)
ds = xr.combine_by_coords([ds,outds])

[########################################] | 100% Completed |  0.1s


In [20]:
ds = ds.compute()
ds

[########################################] | 100% Completed |  6.3s


<xarray.Dataset>
Dimensions:  (region: 3108, year: 68, month: 12)
Coordinates:
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * region   (region) <U5 '21007' '21017' '21031' ... '48171' '55079' '26139'
  * year     (year) int64 1951 1952 1953 1954 1955 ... 2015 2016 2017 2018 2019
Data variables: (12/89)
    dd0      (region, year, month) float64 138.9 174.1 268.0 ... 74.74 67.0
    dd1      (region, year, month) float64 119.5 155.4 239.6 ... 55.27 49.86
    dd10     (region, year, month) float64 16.94 35.32 57.53 ... 0.1125 1.485
    dd11     (region, year, month) float64 12.58 26.96 45.7 ... 0.0129 0.7241
    dd12     (region, year, month) float64 9.045 20.12 35.23 ... 0.0 0.2199
    dd13     (region, year, month) float64 6.006 14.73 26.25 ... 0.0 0.0118
    ...       ...
    time7    (region, year, month) float64 8.208 11.63 17.21 ... 2.076 2.565
    time8    (region, year, month) float64 6.561 10.6 15.58 ... 1.379 1.885
    time9    (region, year, month) float64 5.677 9.73 13.91 ... 0.6512 1.243
    timeM1   (region, year, month) float64 22.38 20.03 30.41 ... 23.12 21.86
    timeM2   (region, year, month) float64 23.67 20.72 30.83 ... 31.0 24.9 24.68
    timeM3   (region, year, month) float64 25.17 21.51 30.95 ... 31.0 26.4 26.58

In [21]:
# https://github.com/pydata/xarray/issues/3476
for v in list(ds.coords.keys()):
    if ds.coords[v].dtype == object:
        ds.coords[v] = ds.coords[v].astype("unicode")

for v in list(ds.variables.keys()):
    if ds[v].dtype == object:
        ds[v] = ds[v].astype("unicode")
        
ds.to_zarr(os.path.join(output_path, output_name+'.zarr'), mode='w')

In [22]:
if csv:
    # ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))
    ds.to_dataframe().to_csv(os.path.join(output_path, output_name+'.csv'))

In [23]:
stata = True
crop = 'soy'
if stata:
    # clim = pd.read_stata("/home3/dth2133/data/SR09/dataSTATA/weather_corn.dta").set_index(['fips', 'year', 'month'])
    # pds = pd.read_stata('/home3/dth2133/data/rescale_prism/rescale_prism.dta').set_index(['fips', 'year', 'month']).drop(columns='index')
    ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr')).to_dataframe()
    ddays = {x:f'dday{x[2:]}C' for x in ds.columns if 'dd' in x and 'M' not in x}
    ddays.update({x:f'ddayMinus{x[3:]}C' for x in ds.columns if 'dd' in x and 'M' in x})
    ddays.update({x:f'time{x[4:]}C' for x in ds.columns if 'time' in x and 'M' not in x})
    ddays.update({x:f'timeMinus{x[5:]}C' for x in ds.columns if 'time' in x and 'M' in x})
    ds.index = ds.index.rename(dict(region='fips'))
    ds = ds.rename(columns=ddays)
    ds = ds.reset_index()
    ds['fips'] = np.int64(ds.fips)
    ds.to_stata(f'/home3/dth2133/data/SR09/dataSTATA/weather_{crop}_era5.dta')

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100